## Homework

> Note: sometimes your answer might not match one of the options exactly. That's fine. 
Select the option that's closest to your solution.

The goal of this homework is to familiarize you with BentoML and how to build and test an ML production service.

## Background

You are a new recruit at ACME corp. Your manager is emailing you about your first assignment.


## Email from your manager

Good morning recruit! It's good to have you here! I have an assignment for you. I have a data scientist that's built
a credit risk model in a jupyter notebook. I need you to run the notebook and save the model with BentoML and see
how big the model is. If it's greater than a certain size, I'm going to have to request additional resources from 
our infra team. Please let me know how big it is.

Thanks,

Mr McManager


## Question 1

* Install BentoML
* What's the version of BentoML you installed?
* Use `--version` to find out

####command line
(ml-zoomcamp) d:\ALEXEY\ML_Zoomcamp\Unit_07\new_model>bentoml --version

####answer
bentoml, version 1.0.7

## Question 2

Run the notebook which contains the xgboost model from module 6 i.e previous module and save the xgboost model with BentoML. To make it easier for you we have prepared this [notebook](https://github.com/alexeygrigorev/mlbookcamp-code/blob/master/course-zoomcamp/07-bentoml-production/code/train.ipynb). 


How big approximately is the saved BentoML model? Size can slightly vary depending on your local development environment.
Choose the size closest to your model.

* 924kb
* 724kb
* 114kb
* 8kb

####command line
(ml-zoomcamp) d:\ALEXEY\ML_Zoomcamp\Unit_07\new_model>bentoml models list

####Result
Tag                                 Module           Size        Creation Time
credit_risk_model:utns5bsq5gs3d7dx  bentoml.xgboost  197.77 KiB  2022-10-20 22:39:54

## Question 3

Say you have the following data that you're sending to your service:

```json
{
  "name": "Tim",
  "age": 37,
  "country": "US",
  "rating": 3.14
}

####text inside service.py

from bentoml.io import JSON

from pydantic import BaseModel

class UserProfile(BaseModel):
  name: str
  age: int
  country: str
  rating: float


## Question 4

We've prepared a model for you that you can import using:

```bash
curl -O https://s3.us-west-2.amazonaws.com/bentoml.com/mlzoomcamp/coolmodel.bentomodel
bentoml models import coolmodel.bentomodel
```

What version of scikit-learn was this model trained with?

* 1.1.1
* 1.1.2
* 1.1.3
* 1.1.4
* 1.1.5

In [ ]:
####run this command
(ml-zoomcamp) d:\ALEXEY\ML_Zoomcamp\Unit_07\new_model>bentoml models get mlzoomcamp_homework:qtzdz3slg6mwwdu5

####results
name: mlzoomcamp_homework
version: qtzdz3slg6mwwdu5
module: bentoml.sklearn
labels: {}
options: {}
metadata: {}
context:
  framework_name: sklearn
  framework_versions:
    scikit-learn: 1.1.1
  bentoml_version: 1.0.7
  python_version: 3.9.12
signatures:
  predict:
    batchable: false
api_version: v1
creation_time: '2022-10-13T20:42:14.411084+00:00'

#answer
"1.1.1"

## Question 5 

Create a bento out of this scikit-learn model. The output type for this endpoint should be `NumpyNdarray()`

Send this array to the Bento:

```
[[6.4,3.5,4.5,1.2]]
```

You can use curl or the Swagger UI. What value does it return? 

* 0
* 1
* 2
* 3

####created the service.py file inside the model folder

import bentoml

from bentoml.io import JSON
from bentoml.io import NumpyNdarray

model_ref = bentoml.sklearn.get("mlzoomcamp_homework:latest")

model_runner = model_ref.to_runner()

svc = bentoml.Service("mlzoomcamp_homework", runners=[model_runner])

@svc.api(input=NumpyNdarray(), output=JSON())
def classify(vector):

    prediction = model_runner.predict.run(vector)
    print(prediction)

    result = prediction[0]

    if result >0.5:
        return { "status": "DECLINED"}
    elif result >0.25:
        return { "status": "MAYBE"}
    else:
        return { "status": "APPROVED"}

####ANSWER: [1]